## Preamble

In [ ]:
! pip install fairseq wandb sacrebleu sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.9/279.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 17.7 MB/s eta 0:00:00
  Prep

## Get baseline metrics

In [3]:
! wget --trust-server-names https://tinyurl.com/nllb200moe54bmetrics -O nllb_54B.csv

--2023-12-04 23:32:45--  https://tinyurl.com/nllb200moe54bmetrics
Resolving tinyurl.com (tinyurl.com)... 104.20.139.65, 172.67.1.225, 104.20.138.65, ...
Connecting to tinyurl.com (tinyurl.com)|104.20.139.65|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dl.fbaipublicfiles.com/large_objects/nllb/models/nllb_200_moe_54b/metrics.csv [following]
--2023-12-04 23:32:45--  https://dl.fbaipublicfiles.com/large_objects/nllb/models/nllb_200_moe_54b/metrics.csv
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.51, 3.162.163.19, 3.162.163.34, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1297810 (1.2M) [text/csv]
Saving to: ‘nllb_54B.csv’

nllb_54B.csv        100%[===================>]   1.24M  5.05MB/s    in 0.2s    

2023-12-04 23:32:46 (5.05 MB/s) - ‘nllb_54B.csv’ saved [1297810/1297810]



In [4]:
import pandas as pd

directions = {"ita_Latn-lij_Latn", "ita_Latn-fur_Latn"}
metrics_nllb_54B = pd.read_csv("nllb_54B.csv")
metrics_nllb_54B.loc[metrics_nllb_54B["direction"].isin(directions)]

,direction,chrf++,spBLEU(spm-200),spBLEU(spm-100)
15534,ita_Latn-fur_Latn,50.2,31.1,32.9
15578,ita_Latn-lij_Latn,45.5,27.5,27.9


## Get data


In [5]:
! wget -O data.zip https://www.dropbox.com/scl/fi/m866661ie28umen9ve0tc/sidetranslation.zip?rlkey=vufpkuch0k4vdqorao9bvk3d7&dl=0
! unzip data.zip

--2023-12-04 23:32:52--  https://www.dropbox.com/scl/fi/m866661ie28umen9ve0tc/sidetranslation.zip?rlkey=vufpkuch0k4vdqorao9bvk3d7
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaf53472a1635e3b9537f9efaf3.dl.dropboxusercontent.com/cd/0/inline/CIxm_5f54n52OnL9yxqz3faP7684AFlUlVtMuOOHCa31jZ4JyUewYYZIE5z6_r3erOE4cB096E5ukw14futbR4S7xOG2HjilccVMOiSmcJQRXCDPHNX4Q7gbMYZmczvOSww/file# [following]
--2023-12-04 23:32:54--  https://ucaf53472a1635e3b9537f9efaf3.dl.dropboxusercontent.com/cd/0/inline/CIxm_5f54n52OnL9yxqz3faP7684AFlUlVtMuOOHCa31jZ4JyUewYYZIE5z6_r3erOE4cB096E5ukw14futbR4S7xOG2HjilccVMOiSmcJQRXCDPHNX4Q7gbMYZmczvOSww/file
Resolving ucaf53472a1635e3b9537f9efaf3.dl.dropboxusercontent.com (ucaf53472a1635e3b9537f9efaf3.dl.dropboxusercontent.com)... 162.125.83.15, 2620:100:6018:15::a27d:30f
Connecting 

## Train

In [ ]:
! LRS=6; EDIM=512; DRP=0.5; ADRP=0.3; RDRP=0.3; LR=1e-3; UF=8; MAXTOK=$((8192*2*8/UF)); fairseq-train "sidetranslation/data_bin" \
  --arch transformer --share-all-embeddings --fp16 \
  --encoder-layers ${LRS} --encoder-ffn-embed-dim $((${EDIM}*4)) --encoder-embed-dim ${EDIM} \
  --decoder-layers ${LRS} --decoder-ffn-embed-dim $((${EDIM}*4)) --decoder-embed-dim ${EDIM} \
  --encoder-attention-heads 8 --decoder-attention-heads 8 \
  --encoder-normalize-before --decoder-normalize-before \
  --attention-dropout ${ADRP} --relu-dropout ${RDRP} --dropout ${DRP} \
  --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
  --lr ${LR} -s ita -t lij \
  --label-smoothing 0.1 \
  --lr-scheduler inverse_sqrt --weight-decay 0 \
  --criterion label_smoothed_cross_entropy --max-update 50000 \
  --warmup-updates 400 --warmup-init-lr '1e-07' \
  --max-tokens ${MAXTOK} --update-freq ${UF} \
  --save-interval 10 --validate-interval 10 --keep-last-epochs 2 --max-tokens-valid ${MAXTOK} \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric --eval-bleu --eval-bleu-remove-bpe sentencepiece --eval-bleu-detok space --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' --eval-bleu-print-samples \
  --save-dir checkpoint

Streaming output truncated to the last 5000 lines.
2023-12-04 20:47:30 | INFO | fairseq_cli.train | Start iterating over samples
2023-12-04 20:47:33 | INFO | fairseq_cli.train | end of epoch 499 (average epoch stats below)
2023-12-04 20:47:33 | INFO | train | epoch 499 | loss 2.305 | nll_loss 0.754 | ppl 1.69 | wps 140977 | ups 1.64 | wpb 85903.8 | bsz 1548.2 | num_updates 1991 | lr 0.000448223 | gnorm 0.178 | loss_scale 4 | train_wall 2 | gb_free 33.4 | wall 2002
2023-12-04 20:47:33 | INFO | fairseq.data.iterators | grouped total_num_itrs = 4
epoch 500:   0% 0/4 [00:00<?, ?it/s]2023-12-04 20:47:33 | INFO | fairseq.trainer | begin training epoch 500
2023-12-04 20:47:33 | INFO | fairseq_cli.train | Start iterating over samples
epoch 500:  75% 3/4 [00:01<00:00,  1.64it/s]2023-12-04 20:47:35 | INFO | fairseq_cli.train | begin validation on "valid" subset

epoch 500 | valid on 'valid' subset:   0% 0/7 [00:00<?, ?it/s]2023-12-04 20:47:37 | INFO | fairseq.tasks.translation | example hypothes

## Generate

In [ ]:
! fairseq-generate "sidetranslation/data_bin" \
  --task translation -s ita -t lij \
  --bpe 'sentencepiece' --sentencepiece-model "sidetranslation/spm/multi.unigram.2k.model" \
  --sacrebleu --scoring sacrebleu --remove-bpe \
  --beam 5 --lenpen 1.0 \
  --gen-subset test \
  --path "checkpoint/checkpoint_best.pt" | grep "^D-" | LC_ALL=C sort -V | cut -f3 > gen.lij

2023-12-04 21:29:27.260101: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 21:29:27.308486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 21:29:27.308557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 21:29:27.308597: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 21:29:27.317226: I tensorflow/core/platform/cpu_feature_g

In [ ]:
! head gen.lij

«Abbia topi de quattro meixi che primma a l'ea de diabetichi e a ô son ciù».
O studio o l'é ancon in fase comensa, comme dito che o l'é deciarou da-o dòtto Ehud Ur, o profescionâ de mediçiña a-a Dalhousie University de Halifax, Nuova Scoçia e diretto do dipartimento clinico e scientifico da Canadian Diabetes Association.
Comme di atri esperti, a l'é scettico che l'é za a poscibilitæ de cuâ o diabete, e l'aggibbo che sti exiti no gh'an za affeto da-o diabete de tipo 1.
A segretaria permanente do Comitato pe-o Nobel pe-a lettiatua de l’Academia svedeise, Sara Danius, into corso de un programma radiofònico e o l’à annonu in sciâ reixe svedeise Sveriges Radios Radio, a l’à annonçiou pubricamente o Comitaçion a l’à annonçiou co-a lettiatua do Bobylan, e pe mezo da communitæ arazzonze do 2016.
O Danius o l'à dischiou: «Ora comme o no l'é scciuppâ de niente. Hofonato e pe-a televixon e inviou a-o seu ciù streito conlaboratô, ch'o l'à respòsto à un mòddo gentile. Pe-o momento, basou e avansou»

In [ ]:
! head sidetranslation/devtest.lij

“Oua gh'emmo di ratti de 4 meixi che no son diabetichi e che in avanti ean diabetichi” o l'à azzonto. 
O dr. Ehud Ur, professô de meixiña a-a Dalhousie University de Halifax, Nova Scotia e prescidente da divixon clinica e scientifica da Canadian Diabetes Association o l'à avvertio che a reçerca a l'é ancon a-i seu primmi pasci.
Pægio de quarche atro esperto, o l'é scettico che o diabete o pòsse ëse cuou, e o l'osserva che ste descoerte chì no gh'an de importansa pe-e gente che gh'an de za o diabete de meña 1.
A-o lunesdì, a Sara Danius, segretäia permanente da Commiscion Nobel pe-a Lettiatua de l'Academia de Sveçia, a l'à annonçiou pubricamente int'un programma radio de l'aradio svedeise in Sveçia che a commiscion, comme a no poeiva arrivâ direttamente a-o Bob Dylan in ponto vittöia do Premmio Nobel pe-a lettiatua 2016, a l'aiva abbandonou i seu sfòrsci pe seghittâlo.
A Danius a l'à dito, “Inte sto momento no femmo ninte. Ò ciammou e ò mandou de email a-i seu ciù streiti conlaboratoî e

In [ ]:
! sacrebleu sidetranslation/devtest.lij -m chrf --chrf-word-order 2 -i gen.lij

{
 "name": "chrF2++",
 "score": 41.5,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:2|space:no|version:2.3.3",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "2",
 "space": "no",
 "version": "2.3.3"
}


In [ ]:
!pwd

/content


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
drive.flush_and_unmount()